<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5-base_5000ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jan 28 14:04:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

In [11]:
# import pandas as pd

# data = pd.read_csv('/content/gdrive/MyDrive/dataset5000pair.csv', usecols=['Thai', 'English'])

# !rm -rf '/content/train.csv'
# !rm -rf '/content/eval.csv'

# from sklearn.model_selection import train_test_split

# train_data, eval_data = train_test_split(data, train_size=0.85, shuffle=True, random_state=1234)
# train_data.to_csv('/content/train.csv', index=False)
# eval_data.to_csv('/content/eval.csv', index=False)

# train_data = pd.read_csv('/content/gdrive/MyDrive/train.csv')
# eval_data = pd.read_csv('/content/gdrive/MyDrive/eval.csv')

# !rm -rf '/content/train.csv'
# !rm -rf '/content/eval.csv'

# import csv

# with open('/content/train.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(train_data)):
#     writer.writerow(['translate thai to english', train_data['Thai'][i], train_data['English'][i]])
#     writer.writerow(['translate english to thai', train_data['English'][i], train_data['Thai'][i]])
# with open('/content/eval.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(eval_data)):
#     writer.writerow(['translate thai to english', eval_data['Thai'][i], eval_data['English'][i]])
#     writer.writerow(['translate english to thai', eval_data['English'][i], eval_data['Thai'][i]])

# pd.read_csv('/content/train.csv')

In [5]:
import pandas as pd

train_data = pd.read_csv('/content/gdrive/MyDrive/train.csv')
train_data

,prefix,input_text,target_text
0,translate thai to english,การศึกษานี้ชี้ให้เห็นว่าปฏิสัมพันธ์ทางสังคมของ...,This study indicates that the social interacti...
1,translate english to thai,This study indicates that the social interacti...,การศึกษานี้ชี้ให้เห็นว่าปฏิสัมพันธ์ทางสังคมของ...
2,translate thai to english,ซึ่งประกอบด้วย 1. ผู้ใช้งานทั่วไป เช่น นักศึกษ...,"Which consists of 1. general users, such as st..."
3,translate english to thai,"Which consists of 1. general users, such as st...",ซึ่งประกอบด้วย 1. ผู้ใช้งานทั่วไป เช่น นักศึกษ...
4,translate thai to english,การทดสอบประสิทธิภาพแสดงระดับความร้อนเฉลี่ยของเ...,The performance test shows the average heat le...
...,...,...,...
8495,translate english to thai,The objective of this study is to select appro...,การศึกษาในครั้งนี้ มีวัตถุประสงค์เพื่อคัดเลือก...
8496,translate thai to english,ผลการทดลองพบว่าการผลิตไบโอดีเซลจากน้ำมันปาล์มบ...,The experimental results found that biodiesel ...
8497,translate english to thai,The experimental results found that biodiesel ...,ผลการทดลองพบว่าการผลิตไบโอดีเซลจากน้ำมันปาล์มบ...
8498,translate thai to english,ผลการวิจัยพบว่าฝาครอบเครื่องมือถูลดข้อบกพร่องเ...,The results of the research showed that the Ru...


In [6]:
eval_data = pd.read_csv('/content/gdrive/MyDrive/eval.csv')
eval_data

,prefix,input_text,target_text
0,translate thai to english,ผู้วิจัยมีวัตถุประสงค์ 3 ประการ 1. เพื่อศึกษาข...,The researcher had 3 objectives 1. to study ge...
1,translate english to thai,The researcher had 3 objectives 1. to study ge...,ผู้วิจัยมีวัตถุประสงค์ 3 ประการ 1. เพื่อศึกษาข...
2,translate thai to english,ค่าเฉลี่ยห้าวันของประสิทธิภาพสูงสุดในแต่ละวันค...,The five days average of the maximum efficienc...
3,translate english to thai,The five days average of the maximum efficienc...,ค่าเฉลี่ยห้าวันของประสิทธิภาพสูงสุดในแต่ละวันค...
4,translate thai to english,จากนั้นวิเคราะห์ผลแบบสอบถามทางด้านความพึงพอใจ ...,"Then, data are analyzed for three parts, which..."
...,...,...,...
1495,translate english to thai,"By the used of Fourier series, it is shown tha...",จากการใช้อนุกรมฟูริเยร์ แสดงให้เห็นว่าสามารถใช...
1496,translate thai to english,ปัจจัยการทดสอบ 3 ปัจจัย ได้แก่ น้ำหนักของกากกา...,Three testing factors were identified weight o...
1497,translate english to thai,Three testing factors were identified weight o...,ปัจจัยการทดสอบ 3 ปัจจัย ได้แก่ น้ำหนักของกากกา...
1498,translate thai to english,ผลการทดลองพบว่าแตงที่ปลูกในดินด่างผสมปุ๋ยหมักอ...,The results showed that the melons grown in an...


In [7]:
import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/gdrive/MyDrive/train.csv').astype(str)
eval_df = pd.read_csv('/content/gdrive/MyDrive/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 3
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

In [ ]:
# !wandb login --relogin

In [8]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/8500 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3707: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

...

Running Epoch 1 of 3:   0%|          | 0/1063 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1063 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

(3189,
 {'global_step': [1063, 2126, 3189],
  'eval_loss': [2.060246671767945, 1.7643990957356515, 1.7058935958020232],
  'train_loss': [1.9271138906478882, 1.60946524143219, 1.688867449760437]})

In [9]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

In [10]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 96
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [13]:
valid = pd.read_csv("/content/gdrive/MyDrive/eval.csv").astype(str)

thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [14]:
english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

Generating outputs:   0%|          | 0/94 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/750 [00:00<?, ?it/s]

Thai to English:  25.788105635101676


Generating outputs:   0%|          | 0/94 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/750 [00:00<?, ?it/s]

English to Thai:  19.75950409278098


In [15]:
from simpletransformers.t5 import T5Model

model = T5Model("mt5", "/content/outputs")
to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']
# to_predict = to_predict[0].split()
predictions = model.predict(to_predict)
for i,info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/9 [00:00<?, ?it/s]

โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ
The mechanical work is to develop a detector system.

และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก
And classification of vegetables from the soil leaves images with detailed

โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน
The using the Json Nano machine to processed the images from the camera in the

ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน
According to the developed a detailed learning model, there was collected a

และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน
And create a variation sign of the gong.

แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)
And the dataset was used to apply to a simulation to detect object in the

ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน
It can be observed that there was a certain variation of kaffir lime

และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์
And when the system detects the fault, it sends 